In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os 
os.chdir('/content/drive/My Drive/Mestrado/2021.1/Redes Neurais Artificiais/RNA/')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import math

from models import utils, metrics, plots

from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import random
import seaborn as sns
from copy import deepcopy

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [53]:
data = load_iris()
df_iris = pd.DataFrame(data.data, columns=data.feature_names)
df_iris['target'] = pd.Series(data.target)

for col in df_iris.columns:
    if col != 'target':
        df_iris = utils.normalize_col(df_iris, col)

In [54]:
def process(y):
    if y == 0:
        return [1, 0, 0]
    if y == 1:
        return [0, 1, 0]
    if y == 2:
        return [0, 0, 1]

def back(y):
    if y == [1, 0, 0]:
        return 0
    if y == [0, 1, 0]:
        return 1
    if y == [0, 0, 1]:
        return 2

In [55]:
X_train, y_train, X_test, y_test = utils.split_train_test(df_iris, 'target', train_size=0.8, stratify=True)

df_train = X_train.copy()
df_train['target'] = [process(y) for y in y_train]

df_test = X_test.copy()
df_test['target'] = [process(y) for y in y_test]

In [56]:
def get_optimal_weigths(df_train, col_target='target'):
    X = np.asmatrix(df_train.drop([col_target], axis=1).values)
    D = np.asmatrix(np.array(df_train[col_target].tolist()))

    W = ((X.T * X).I * X.T * D)

    return W

In [58]:
def predict_row(row, W, qt_classes=3):
  
    result = (row.tolist() * W).tolist()[0]
    df = pd.DataFrame(zip([process(i) for i in range(qt_classes)], result), columns=['class', 'prob'])
    output = df.sort_values(by='prob', ascending=False).iloc[0][0]
    return output


def predict_dataframe(df, W, qt_classes=3):

    list_result = []
    for idx, _ in df.iterrows():
        list_result.append(predict_row(df.iloc[idx], W, qt_classes=qt_classes))

    return list_result

In [59]:
W = get_optimal_weigths(df_train.copy(), col_target='target')

In [60]:
df_test['predict'] = predict_dataframe(df_test.drop(['target'], axis=1).copy(), W, qt_classes=3)

In [61]:
df_test.head(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,predict
0,0.138889,0.416667,0.067797,0.083333,"[1, 0, 0]","[1, 0, 0]"
1,0.305556,0.583333,0.084746,0.125000,"[1, 0, 0]","[1, 0, 0]"


In [62]:
y_true = df_test['target'].apply(lambda x: back(x))
y_pred = df_test['predict'].apply(lambda x: back(x))

In [63]:
metrics.taxa_acerto(y_true, y_pred)

0.7333333333333333

In [64]:
df_train['predict'] = predict_dataframe(df_train.drop(['target'], axis=1).copy(), W, qt_classes=3)

In [65]:
y_true = df_train['target'].apply(lambda x: back(x))
y_pred = df_train['predict'].apply(lambda x: back(x))

In [66]:
metrics.taxa_acerto(y_true, y_pred)

0.7833333333333333